# Sprint 3

In de [derde sprint](https://trello.com/b/EUxZhLuE/sprint-3-milestone-1-27-nov) werken we aan de volgende taken:

1. [Helper Functions](http://localhost:8888/notebooks/Sprint%203.ipynb#Helper-Functions)
2. [Top Layer van het Systeem](http://localhost:8888/notebooks/Sprint%203.ipynb#1.-Top-Layer-van-het-Systeem) ~Jim
3. [Feature score berekenen van woorden, feature vectors maken](http://localhost:8888/notebooks/Sprint%203.ipynb#2.-Feature-score-berekenen-van-woorden,-feature-vectors-maken) ~Stefan
4. [Fuzzy Logic Toolbox implementeren]() ~Peter

### 1. Helper Functions

In [Sprint 2](http://localhost:8888/notebooks/Sprint%202.ipynb) hebben we methodes gemaakt om woorden te tellen en een intersectie tussen woordenlijsten uit te voeren. Deze methodes kunnen worden gebruikt bij de taken.

In [1]:
import csv
import nltk
import string

nltk.download('punkt')
nltk.download('stopwords')

from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

def tokenize(body):
    tokens = word_tokenize(body)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    porter = PorterStemmer()
    words = [porter.stem(word) for word in words]
    return words

def read_txt(filepath):
    with open(filepath, 'r') as t:
        body = t.read()
        return tokenize(body)
    
def read_csv(filepath):
    with open(filepath, 'r') as c:
        reader = csv.reader(c, delimiter=',')
        for row in reader:
            return row

def generate_csv_from_array(filename, array):
    with open("res/" + filename + ".csv", 'w', newline='') as c:
        writer = csv.writer(c, delimiter=',')
        writer.writerow(array)
        
def intersection(array1, array2):
    """Returns a generator, use next(generator)"""
    return (i for i in array1 if i in array2)

[nltk_data] Downloading package punkt to /home/stefan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/stefan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 2. Top Layer van het Systeem
    ~Jim Kamans

Er moet een systeem geschreven worden (gedurende alle sprints) die de basis vormt voor het classificeren van meerdere emails, waarbij een overzicht van de classificatie en analyse van de correct en fout geclassificeerde emails wordt getoond.

#### 2.1 Inlezen meerdere emails uit folder

... leg uit wat je doet in deze stap ...

### 3. Feature score berekenen van woorden, feature vectors maken

    ~Stefan Schenk
    
Woorden in een corpus moeten extra features krijgen, namelijk de scores van de mate waarbij ieder woord bij een feature past. Daarna moet de algehele score van alle woorden bij elkaar worden berekend. Dit zullen de inputs zijn voor het Fuzzy Logic Systeem.

#### 3.1 Generating Word List

... leg uit wat je doet in deze stap ...

In [5]:
# Maakt een word_list die simpelweg alle woorden bevat die 
# een evental aan karakters hebben
email     = read_txt('res/email.txt')
word_list = [x for x in sorted(set(email)) if len(x) % 2 == 0]
generate_csv_from_array("word_list", word_list)

# Maakt een feature_list met woorden die met een c beginnen
starts_with_c = [x for x in word_list if x[0] == 'c']
generate_csv_from_array("starts_with_c", starts_with_c)

# Maakt een feature_list met woorden die met een m beginnen
starts_with_m = [x for x in word_list if x[0] == 'm']
generate_csv_from_array("starts_with_m", starts_with_m)

# print(email)

### 4. Fuzzy Logic Toolbox implementeren

    ~Peter Heemskerk
    
Er moet een systeem geschreven worden die de basis vormt voor het classificeren van meerdere emails, waarbij een overzicht van de classificatie en analyse van de correct en fout geclassificeerde emails wordt getoond.

#### 4.1 Klassen Toevoegen Uit De Oefen Lab

... leg uit wat je doet in deze stap ...